In [1]:

def test_split(index, value, dataset):
    left, right = list(), list()
    for row in dataset:
        if row[index] < value:
            left.append(row)
        else:
            right.append(row)
    return left, right


def gini_index(groups, classes):
    n_instances = float(sum([len(group) for group in groups]))
    gini = 0.0
    for group in groups:
        size = float(len(group))
        if size == 0:
            continue
        score = 0.0
        for class_val in classes:
            p = [row[-1] for row in group].count(class_val) / size
            score += p * p
        gini += (1.0 - score) * (size / n_instances)
    return gini


def get_split(dataset):
    class_values = list(set(row[-1] for row in dataset))
    b_index, b_value, b_score, b_groups = 999, 999, 999, None
    for index in range(len(dataset[0])-1):
        for row in dataset:
            groups = test_split(index, row[index], dataset)
            gini = gini_index(groups, class_values)
            print('data%d < %.3f Gini=%.3f' % ((index+1), row[index], gini))
            if gini < b_score:
                b_index, b_value, b_score, b_groups = index, row[index], gini, groups
    return {'index':b_index, 'value':b_value, 'groups':b_groups}

dataset = [[60,65,23.4,1],
    [40,15,36.5,0],
    [54,75,77.2,1],
    [57,40,25.4,0],
    [26,100,99,1],
    [84,60,26,1],
    [23,56,43.2,0],
    [53,90,22.6,1],
    [92,23,23.2,1]]

split = get_split(dataset)
print('Split: [data%d < %.3f]' % ((split['index']+1), split['value']))

data1 < 60.000 Gini=0.333
data1 < 40.000 Gini=0.429
data1 < 54.000 Gini=0.400
data1 < 57.000 Gini=0.433
data1 < 26.000 Gini=0.333
data1 < 84.000 Gini=0.381
data1 < 23.000 Gini=0.444
data1 < 53.000 Gini=0.333
data1 < 92.000 Gini=0.417
data2 < 65.000 Gini=0.267
data2 < 15.000 Gini=0.444
data2 < 75.000 Gini=0.333
data2 < 40.000 Gini=0.429
data2 < 100.000 Gini=0.417
data2 < 60.000 Gini=0.167
data2 < 56.000 Gini=0.333
data2 < 90.000 Gini=0.381
data2 < 23.000 Gini=0.333
data3 < 23.400 Gini=0.381
data3 < 36.500 Gini=0.400
data3 < 77.200 Gini=0.381
data3 < 25.400 Gini=0.333
data3 < 99.000 Gini=0.417
data3 < 26.000 Gini=0.433
data3 < 43.200 Gini=0.444
data3 < 22.600 Gini=0.444
data3 < 23.200 Gini=0.417
Split: [data2 < 60.000]


In [2]:

def to_terminal(group):
    outcomes = [row[-1] for row in group]
    return max(set(outcomes), key=outcomes.count)

def split(node, max_depth, min_size, depth):
    left, right = node['groups']
    del(node['groups'])
    if not left or not right:
        node['left'] = node['right'] = to_terminal(left + right)
        return
    if depth >= max_depth:
        node['left'], node['right'] = to_terminal(left), to_terminal(right)
        return
    if len(left) <= min_size:
        node['left'] = to_terminal(left)
    else:
        node['left'] = get_split(left)
        split(node['left'], max_depth, min_size, depth+1)
    if len(right) <= min_size:
        node['right'] = to_terminal(right)
    else:
        node['right'] = get_split(right)
        split(node['right'], max_depth, min_size, depth+1)


def build_tree(train, max_depth, min_size):
    root = get_split(train)
    split(root, max_depth, min_size, 1)
    return root


def print_tree(node, depth=0):
    if isinstance(node, dict):
        print('%s[data%d < %.3f]' % ((depth*' ', (node['index']+1), node['value'])))
        print_tree(node['left'], depth+1)
        print_tree(node['right'], depth+1)
    else:
        print('%s[%s]' % ((depth*' ', node)))

tree = build_tree(dataset, 3, 3)
print_tree(tree)

data1 < 60.000 Gini=0.333
data1 < 40.000 Gini=0.429
data1 < 54.000 Gini=0.400
data1 < 57.000 Gini=0.433
data1 < 26.000 Gini=0.333
data1 < 84.000 Gini=0.381
data1 < 23.000 Gini=0.444
data1 < 53.000 Gini=0.333
data1 < 92.000 Gini=0.417
data2 < 65.000 Gini=0.267
data2 < 15.000 Gini=0.444
data2 < 75.000 Gini=0.333
data2 < 40.000 Gini=0.429
data2 < 100.000 Gini=0.417
data2 < 60.000 Gini=0.167
data2 < 56.000 Gini=0.333
data2 < 90.000 Gini=0.381
data2 < 23.000 Gini=0.333
data3 < 23.400 Gini=0.381
data3 < 36.500 Gini=0.400
data3 < 77.200 Gini=0.381
data3 < 25.400 Gini=0.333
data3 < 99.000 Gini=0.417
data3 < 26.000 Gini=0.433
data3 < 43.200 Gini=0.444
data3 < 22.600 Gini=0.444
data3 < 23.200 Gini=0.417
data1 < 40.000 Gini=0.333
data1 < 57.000 Gini=0.250
data1 < 23.000 Gini=0.375
data1 < 92.000 Gini=0.000
data2 < 15.000 Gini=0.375
data2 < 40.000 Gini=0.250
data2 < 56.000 Gini=0.333
data2 < 23.000 Gini=0.333
data3 < 36.500 Gini=0.250
data3 < 25.400 Gini=0.000
data3 < 43.200 Gini=0.333
data3 < 23.

In [3]:

def predict(node, row):
    if row[node['index']] < node['value']:
        if isinstance(node['left'], dict):
            return predict(node['left'], row)
        else:
            return node['left']
    else:
        if isinstance(node['right'], dict):
            return predict(node['right'], row)
        else:
            return node['right']


stump = {'index': 0, 'right': 1, 'value': 0, 'left': 0}
for row in dataset:
    prediction = predict(stump, row)
    print('Expected=%d, Got=%d' % (row[-1], prediction))

Expected=1, Got=1
Expected=0, Got=1
Expected=1, Got=1
Expected=0, Got=1
Expected=1, Got=1
Expected=1, Got=1
Expected=0, Got=1
Expected=1, Got=1
Expected=1, Got=1
